# Postgres Data Exploration
Use the helper functions below to inspect tables in the project database.


**Setup**
- Ensure `.env` contains a valid `DATABASE_URL` pointing at Postgres.
- Update `table_to_preview` or the custom SQL cells to target the tables you want to inspect.


In [ ]:
from __future__ import annotations

import os
from pathlib import Path
from typing import Any

import pandas as pd
from dotenv import load_dotenv
from sqlalchemy import MetaData, Table, create_engine, select, text
from sqlalchemy.engine import Engine

PROJECT_ROOT = Path("..").resolve()
load_dotenv(PROJECT_ROOT / ".env")

DATABASE_URL = os.environ["DATABASE_URL"]
engine: Engine = create_engine(DATABASE_URL, future=True)
metadata = MetaData()

print(f"Engine ready for {engine.url.render_as_string(hide_password=True)}")


In [ ]:
def run_query(sql: str, params: dict[str, Any] | None = None) -> pd.DataFrame:
    """Execute a raw SQL query and return a DataFrame."""
    query = text(sql)
    with engine.connect() as connection:
        return pd.read_sql(query, connection, params=params)


def preview_table(table_identifier: str, limit: int = 10) -> pd.DataFrame:
    """Load a table by name (optionally schema-qualified) for a quick look."""
    schema: str | None = None
    table_name = table_identifier
    if "." in table_identifier:
        schema, table_name = table_identifier.split(".", 1)

    table_obj = Table(table_name, metadata, autoload_with=engine, schema=schema)
    statement = select(table_obj).limit(limit)
    with engine.connect() as connection:
        return pd.read_sql(statement, connection)


In [ ]:
pd.options.display.max_rows = 20
pd.options.display.max_columns = None
pd.options.display.width = 0


In [ ]:
tables = run_query(
    """
    SELECT table_schema, table_name
    FROM information_schema.tables
    WHERE table_type = 'BASE TABLE'
      AND table_schema NOT IN ('pg_catalog', 'information_schema')
    ORDER BY table_schema, table_name
    """
)
tables


In [ ]:
# Replace with the table you want to preview, e.g. "public.players" or "public.games"
table_to_preview = "public.players"
preview_table(table_to_preview, limit=25)


In [ ]:
# Customize the SQL below for deeper exploration.
custom_sql = """
-- Example: row counts per table
SELECT table_schema,
       table_name,
       pg_total_relation_size(format('%s.%s', table_schema, table_name)) AS bytes
FROM information_schema.tables
WHERE table_type = 'BASE TABLE'
  AND table_schema NOT IN ('pg_catalog', 'information_schema')
ORDER BY bytes DESC
LIMIT 20
"""

run_query(custom_sql)
